# **[IntoAI] WEEK4 : Probablity**

##### **불확실성을 수치로 표현된 확신도를 이용해서 길들이는 방법을 살펴보며 불확실성하에서 확률론의 법칙들에 의거한 추론에 쓰이는 효율적인 네트워크 모형을 구축하는 방법을 알아본다.**

### **목차**
1. 확률 분포
2. 지식 베이스 : PropKB & 웜퍼스 세계
3. 지식 기반 에이전트와 추론
4. 분해 증명
5. 순방향과 역방향 연쇄


*인공지능 : 현대적 접근방식(제4판)* 도서를 기반으로 작성되었으며 https://github.com/aimacode/aima-python 을 참고하였습니다.
* **Chapter12 | 불확실성의 정량화, Chapter13 | 확률적 추론** 의 내용을 포함하고 있습니다.
* 코드와 이론에 대한 자세한 설명은 (블로그 링크)를 참고하세요.
* **Google Colab**을 기반으로 작동하도록 제작되었습니다.
* notebook을 실행하기 위해서는 **probablity.py, utils.py** 모듈이 포함되어야 합니다.


### **0. 기본 환경 세팅**

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 모듈이 들어있는 디렉토리로 이동하기
%cd drive/MyDrive/TA/인공지능 입문/SCS4033_Introduce-to-AI/Week 3

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/TA/인공지능 입문/SCS4033_Introduce-to-AI/Week 3'
/content/drive/MyDrive/TA/인공지능 입문/SCS4033_Introduce-to-AI/Week 3


In [17]:
from utils import *
from probability import *
from notebook import psource

### **1. 확률 분포**

확률 분포란, 확률 변수가 가질 수 있는 각각의 값에 어떤 확률이 배정되는 지를 결정한다. 이번 실습에서는 이런 확률 변수를 쉽게 정의하고 나타낼 수 있는 클래스를 활용하여 다양한 경우를 탐색해보고자 한다.

우선, ProbDist 클래스를 통해서 이산 확률 분포를 지정해보자. 랜덤 변수의 이름을 지정한 다음 확률을 할당할 수 있다. 딕셔너리와 유사하게 생각하면 된다. 

In [18]:
psource(ProbDist)

In [24]:
p = ProbDist('Flip')
p['H'], p['T'] = 0.25, 0.75
print(p['T'])
print(p.var_name)

0.75
Flip


`varname`을 지정해주지 않으면 **?**로 기본설정된다.
`freqs`를 통해서 딕셔너리 형태로 `{랜덤 변수 : 확률}`의 형태로 클래스를 정의할 수 있다.
그리고 `확률`란에 value로 입력된 값들을 모두 `normalized`되어 입력된 모든 value들의 합이 1인 형태로 자동 정규화된다. 

In [26]:
p = ProbDist(freq={'low': 125, 'medium': 375, 'high': 500})
print(p.var_name)

?


In [27]:
(p['low'], p['medium'], p['high'])

(0.125, 0.375, 0.5)

확률 분포 내의 모든 값을 개별적으로도 확인할 수 있다. 즉, unique한 랜덤 변수들의 값을 `values`를 통해서 확인할 수 있으며 이는 `__setitem__` 메서드에서 확인 가능하다 

In [28]:
p.values

['low', 'medium', 'high']

값들이 한번에 입력되면 자동으로 정규화를 진행하지만, 그렇지 않고 하나씩 값이 추가되면 자동으로 정규화를 하지 않는 문제가 있다. 이는 `normalize` 메서드를 활용하여 해결할 수 있다.

In [29]:
p = ProbDist('Y')
p['Cat'] = 50
p['Dog'] = 114
p['Mice'] = 64
(p['Cat'], p['Dog'], p['Mice'])

(50, 114, 64)

In [30]:
p.normalize()
(p['Cat'], p['Dog'], p['Mice'])

(0.21929824561403508, 0.5, 0.2807017543859649)

근사 값을 `show_approx` 메서드를 통해 보기 좋게 확인할 수 있다.

In [31]:
p.show_approx()

'Cat: 0.219, Dog: 0.5, Mice: 0.281'